In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [8]:
x1 = torch.randn((1, 512, 32, 32))
x2 = torch.randn((1, 512, 32, 32))
y = (torch.randn((1, 32, 32)) > 0.5).float()

diff = torch.abs(x1 - x2)

dist_sq = torch.pow(diff + 1e-6, 2).sum(dim=1)  # 加一个微小扰动, sum(dim=1)沿通道方向求和，故输出尺寸为(1,32,32)

dist = torch.sqrt(dist_sq)  # 求距离(L2)

mdist_pos = torch.clamp(0.3-dist, min=0.0)  # m1 = 0.3, unchanged
mdist_neg = torch.clamp(dist-2.2, min=0.0)  # m2 = 2.2, changed

w1 = 1 / 0.147
w2 = 1 / (1 - 0.147)
loss_pos = w2 * ((1 - y) * (mdist_pos.pow(2)))  # 矩阵乘  w2小，使网络较为不关注unchanged pairs
loss_neg = w1 * (y * (mdist_neg.pow(2)))        # 矩阵乘  w1大，使网络重点关注changed pairs

loss = torch.mean(loss_pos + loss_neg)          # 原代码没加权
loss

tensor(1947.2969)

In [122]:
# 原始label可能尺寸小于feature，故插值构成新的label
def rz_label(label, size):
    gt_e = torch.unsqueeze(label, dim=1)
    interp = nn.functional.interpolate(gt_e, (size[0],size[1]), mode='bilinear', align_corners=True)
    gt_rz = torch.squeeze(interp, dim=1)
    return gt_rz

label = (torch.randn((1,4,4)) > 0.5).float()
label, rz_label(label, (8,8))

(tensor([[[0., 0., 0., 1.],
          [0., 0., 0., 1.],
          [1., 0., 0., 1.],
          [1., 1., 0., 0.]]]),
 tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1429, 0.5714, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1429, 0.5714, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1429, 0.5714, 1.0000],
          [0.2857, 0.1633, 0.0408, 0.0000, 0.0000, 0.1429, 0.5714, 1.0000],
          [0.7143, 0.4082, 0.1020, 0.0000, 0.0000, 0.1429, 0.5714, 1.0000],
          [1.0000, 0.6327, 0.2653, 0.1020, 0.0408, 0.1224, 0.4898, 0.8571],
          [1.0000, 0.8163, 0.6327, 0.4082, 0.1633, 0.0612, 0.2449, 0.4286],
          [1.0000, 1.0000, 1.0000, 0.7143, 0.2857, 0.0000, 0.0000, 0.0000]]]))